In [4]:
import os
import shutil
import yaml
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#based on https://docs.ultralytics.com/datasets/detect/

In [ ]:
basepath = './kogni/kogni'

dir_items = os.listdir(basepath)
dir_items

['classes.txt',
 'jpg-os1-filip',
 'jpg-os1-filip-new',
 'jpg-os2-jan',
 'jpg-os2-jan-new',
 'jpg-os3-mikolaj',
 'jpg-os3-mikolaj-new',
 'jpg-os4-norbert',
 'jpg-os4-norbert-new',
 'jpg-os5-oskar',
 'jpg-os5-oskar-new']

In [ ]:
new_dest_path = "whole_dataset"
new_labels_dataset = f"{new_dest_path}/labels"
new_imgs_dataset = f"{new_dest_path}/images"

os.mkdir(new_dest_path)
os.mkdir(new_labels_dataset)
os.mkdir(new_imgs_dataset)

In [ ]:
for item in dir_items:
    if os.path.isdir(subdir := os.path.join(basepath, item)):
        subdir_entries = os.listdir(subdir)
        if len(potential_label_dirs := [entry for entry in subdir_entries if os.path.isdir(os.path.join(subdir, entry))]) > 0:
            labels_dir = os.path.join(subdir, potential_label_dirs[0])
            print(labels_dir)
            label_files = os.listdir(labels_dir)
            for l_file_txt in label_files:
                shutil.copy(os.path.join(labels_dir, l_file_txt), new_labels_dataset)
                img_file_jpg = l_file_txt.replace("txt", "jpg")
                shutil.copy(os.path.join(subdir, img_file_jpg), new_imgs_dataset)

./kogni/kogni\jpg-os1-filip\labels_dron_2024-05-21-08-41-50
./kogni/kogni\jpg-os2-jan\labels
./kogni/kogni\jpg-os3-mikolaj\labels
./kogni/kogni\jpg-os3-mikolaj-new\labels
./kogni/kogni\jpg-os5-oskar\labels
./kogni/kogni\jpg-os5-oskar-new\labels


In [ ]:
print(f"number of .txt labels = {len(os.listdir(new_labels_dataset))}")
print(f"number of .jpg images = {len(os.listdir(new_imgs_dataset))}")

number of .txt labels = 614
number of .jpg images = 614


In [ ]:
train, val = train_test_split(os.listdir(new_labels_dataset), test_size=0.2, random_state=42)

In [ ]:
print(f"train size = {len(train)}")
print(f"validation size = {len(val)}")

train size = 491
validation size = 123


In [ ]:
train[:10]

['DJI_0655_00141.txt',
 'DJI_0655_00150.txt',
 'DJI_0848_00007.txt',
 'DJI_0918_00001.txt',
 'DJI_0910_00074.txt',
 'DJI_0900_00007.txt',
 'DJI_0655_00137.txt',
 'DJI_0655_00044.txt',
 'DJI_0732_00004.txt',
 'DJI_0859_00078.txt']

In [ ]:
final_dataset_path = "./dataset"
images_dataset = f"{final_dataset_path}/images"
labels_dataset = f"{final_dataset_path}/labels"

os.mkdir(final_dataset_path)
os.mkdir(images_dataset)
os.mkdir(labels_dataset)

for subdir in ['train', 'val', 'test']: #test dataset ignored for now
    os.mkdir(f"{images_dataset}/{subdir}")
    os.mkdir(f"{labels_dataset}/{subdir}")

In [ ]:
f"{images_dataset}/{subdir}"

'./dataset/images/test'

In [ ]:
for filename in train:
    shutil.copy(os.path.join(new_labels_dataset, filename), './dataset/labels/train')
    img_file_jpg = filename.replace("txt", "jpg")
    shutil.copy(os.path.join(new_imgs_dataset, img_file_jpg), './dataset/images/train')

In [ ]:
for filename in val:
    shutil.copy(os.path.join(new_labels_dataset, filename), './dataset/labels/val')
    img_file_jpg = filename.replace("txt", "jpg")
    shutil.copy(os.path.join(new_imgs_dataset, img_file_jpg), './dataset/images/val')

In [ ]:
i = 0
classes_dict = {}
with open('./classes.txt', 'r') as file:
    for line in file:
        classes_dict[i] = line.strip()
        i+=1

print(classes_dict)

{0: 'pedestrian', 1: 'bicycle', 2: 'scooter', 3: 'tram', 4: 'bus', 5: 'car', 6: 'delivery', 7: 'truck', 8: 'motorbike'}


In [ ]:
yaml_config = {
    'path': './dataset',
    'train': 'images/train',
    'val': 'images/val',
    # 'test': 'images/test'
    'names': classes_dict,
}

In [ ]:
yaml_config

{'path': './dataset',
 'train': 'images/train',
 'val': 'images/val',
 'names': {0: 'pedestrian',
  1: 'bicycle',
  2: 'scooter',
  3: 'tram',
  4: 'bus',
  5: 'car',
  6: 'delivery',
  7: 'truck',
  8: 'motorbike'}}

In [ ]:
with open('dataset.yaml', 'w') as file:
    yaml.dump(yaml_config, file)

In [ ]:
! unzip dataset.zip -d datasets/dataset

In [ ]:
! pip install ultralytics

In [ ]:
# for training in colab dataset should be unpacked in content/datasets directory

In [11]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

In [15]:
results = model.train(data="dataset.yaml", epochs=50, imgsz=640)

Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|██████████| 755k/755k [00:00<00:00, 21.3MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

      50/50      2.54G      1.242     0.8036     0.9268        193        640: 100%|██████████| 31/31 [00:25<00:00,  1.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.51it/s]

                   all        123       1423      0.613      0.484      0.482      0.298



50 epochs completed in 0.488 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:10<00:00,  2.69s/it]


                   all        123       1423      0.611      0.496      0.493      0.303
            pedestrian         54        488      0.421      0.266      0.251     0.0848
               bicycle         23         29      0.273      0.207      0.182     0.0638
               scooter          1          1          1          0          0          0
                  tram         14         25      0.528        0.8      0.671      0.482
                   bus         26         54      0.609      0.889      0.855      0.646
                   car        103        720      0.757       0.81      0.826       0.48
              delivery         41         79      0.501      0.709      0.631       0.42
                 truck         13         17      0.593      0.344      0.357      0.274
             motorbike         10         10      0.814      0.441      0.664      0.274
Speed: 0.5ms preprocess, 2.7ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/detect

In [16]:
model.export()

Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3007403 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train4/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 13, 8400) (5.9 MB)

TorchScript: starting export with torch 2.3.0+cu121...
TorchScript: export success ✅ 2.1s, saved as 'runs/detect/train4/weights/best.torchscript' (11.9 MB)

Export complete (3.8s)
Results saved to /content/runs/detect/train4/weights
Predict:         yolo predict task=detect model=runs/detect/train4/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train4/weights/best.torchscript imgsz=640 data=dataset.yaml  
Visualize:       https://netron.app


'runs/detect/train4/weights/best.torchscript'

In [22]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [27]:
model = YOLO('yolov8n.yaml').load('runs/detect/train4/weights/best.pt')

Transferred 319/355 items from pretrained weights


In [28]:
 results = model('./mini_test')


image 1/5 /content/mini_test/DJI_0655_00124.jpg: 384x640 (no detections), 177.4ms
image 2/5 /content/mini_test/DJI_0846_00042.jpg: 384x640 (no detections), 7.6ms
image 3/5 /content/mini_test/DJI_0906_00053.jpg: 384x640 (no detections), 7.5ms
image 4/5 /content/mini_test/DJI_0918_00085.jpg: 384x640 (no detections), 29.2ms
image 5/5 /content/mini_test/DJI_0953_00039.jpg: 384x640 (no detections), 9.0ms
Speed: 4.6ms preprocess, 46.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


In [33]:
results[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))